In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params): #query database
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt): #regex
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [3]:
# parameters
customer_id = '115'
formatted_attribute = 'Preferences'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Timing Belt"""

value='%n/a%'
customer_name='%lovelyskin%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
metric = ' Inches'

range_string = """
 0 - 0.5000 Inches
 0 - 10 Inches
 0.5001 - 1.0000 Inches
 0 - 5 Inches
 1.0001 - 1.5000 Inches
 11 - 15 Inches
 11 - 20 Inches
 1.5001 - 2.0000 Inches
 16 - 20 Inches
 2.0001 - 2.5000 Inches
 21 - 25 Inches
 21 - 30 Inches
 2.5001 - 3.0000 Inches
 26 - 30 Inches
 3.0001 - 3.5000 Inches
 31 - 40 Inches
 3.5001 - 4.0000 Inches
 4.0001 - 4.5000 Inches
 41 - 50 Inches
 4.5001 - 5.0000 Inches
 6 - 10 Inches
 Greater Than 30 Inches
 Greater than 5.0000 Inches
 Greater than 50 Inches
 """

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [6]:
df = query_from_file(file_name='./query/Curated_lovelyskin.sql', params=params)
pref=df[df['value'].astype(str)=='n/a']
len(pref)

9187

In [7]:
# frizz free
# UV shield
# Oil-Free
# hydrates skin
# hydrates hair
# neutralizes free radicals
# removes impurities
# increases the production of free fatty acids
# flake-free 
# # fragrance-free
# # Free of sodium chloride
# # contains/free of  Sodium hyaluronate?
# Smudge proof/resistant
# shine-free finish--promotes shine

# acne and blemishes
# prevents water loss
# prevent clogged pores
# prevent wrinkles
# prevents future breakouts
# prevent cellular oxidation
# preventing future tissue damage
# prevent breakage

# calm skin
# absorb completely
# optimal exfoliation
# safeguard against harsh external factors
# prevent dehydration
# radiance to your complexion
# minimize signs of aging
# ultra-lubricating
# remove dirt, oil and debris
# enhance peel results
# impurities
# detoxify
# limit skin imperfections
# anti-ageing
# revitalize the skin
# tighten and firm
# brighten, retexturize, soothe irritation
# rich in amino acid

In [8]:
# (amino.?acid)|(anti.?ageing)|(detoxify)|(peel)|(remove dirt)|(ultra-lubricating)|(radiance)|(dehydration)|(calm skin)|(absorb)|(exfoliation)|(breakage)|(tissue damage)|(cellular oxidation)|(wrinkles)|(pores)|(water loss)|(breakouts)|(blemishes)|(acne)|(shine.?free.?finish)|(Smudge proof)|(flake.?free)|(fatty acids)|(free radicals)|(hydrates)|(Oil.?Free)|(UV)|(frizz)|(Free.*sodium.?chloride)


In [9]:
# (irritation)|(tighten)|(revitalize.*skin)|(skin.?imperfections)|

In [10]:
# soy proteins, Keratin Complex Vanilla, Botanical extracts, Hand crafted, Hyaluronic acid, waterproof liner, cocoa and shea butters, Hydrating Lipstick, hydrating, Free of sodium chloride, free of sulfates, prevents dandruff,  pain-free,  Silicone-alternative,  breaking up plaque,  preventing tartar buildup,  lifting stains,hyaluronic acid   

In [11]:
# stimulates collagen production, 

In [32]:
regex_pattern = r"(?i)(Vegan)|(Organic)|(Talc)|(Sulfate)|(Soap)|(f?r?e?e?.?Silicone.f?r?e?e?)|(Preservative)|(Phthalate)|(ph.?balanced)|(Paraben)|(Oil.?Free)|(Non.?Toxic)|(Non.?GMO)|(Mica.?Free)|(Latex.?Free)|(Kosher)|(Hypoallergenic)|(Gluten.?free)|(Fragrance.?free)|(Dye.?free)|(Cruelty.?free)|(Chemical.?free)|(Aluminum.?free)|(All.?Natural)|(Alcohol)|(Silicone.?alternative)|(Paraben)|(Free.*sodium chloride)|(free.*sulfates)|(Compact)|(UV)|(peel)|(dry.?shampoo)|(quick drying)|(travel-size)()"                      
pref['matches'] = pref['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# x=pref[pref['matches'].astype(str)!='[]']
# print("This is the number of matches" +f' {len(x)}')
y=pref[pref['matches'].astype(str)=='[]']
print(len(pref)-len(y))

regex_pattern_current = r"(?i)(hydroquinone)|(tinted sunscreen)|(bed.?head)|( vitamin enhanced omega oil)|(benzoyl peroxide face)|(Hemp.?Seed.?Oil)|()"                      
y['matches'] = y['long_desc'].apply(lambda x: re_extract(regex_pattern_current, x))
z=y[y['matches'].astype(str)=='[]']
print(len(y)-len(z))

# regex_pattern_new = r"(?i)(soy proteins)|(Keratin)|(Botanical)|(Hand.?crafted)|(Hyaluronic)|(waterproof)|(cocoa)|(shea butters)|(Hydrat)|(Paraben)|(Free.*sodium chloride)|(free.*sulfates)|(pain.?free)|(Silicone.?alternative)|(plaque)|(tartar)|(lifting stains)|(hyaluronic acid)|(natural.?look)|(memory.?enhancing)()"                      
# z['matches'] = z['long_desc'].apply(lambda x: re_extract(regex_pattern_new, x))
# A=z[z['matches'].astype(str)=='[]']
# print(len(A))
# A[0:500]

1702
118


In [34]:
z[2500:3000]

attributes snake_case_name external_id  product_id  \
8585  Preferences     preferences      S29502   650106351   
8586  Preferences     preferences      S29503   650106352   
8587  Preferences     preferences      S29504   650106357   
8588  Preferences     preferences      S29505   650106358   
8589  Preferences     preferences      S29506   650106359   
8590  Preferences     preferences      S29507   650106360   
8591  Preferences     preferences      S29508   650106361   
8592  Preferences     preferences      S29509   650106362   
8593  Preferences     preferences      S30076   650106363   
8594  Preferences     preferences      S29419   650106365   
8595  Preferences     preferences      S29418   650106367   
8596  Preferences     preferences      S29481   650106375   
8600  Preferences     preferences      S29441   650106386   
8601  Preferences     preferences      S29448   650106389   
8602  Preferences     preferences      S29445   650106391   
8604  Preferences     preferences      S29427   650106409   
8605  Preferences     preferences      S29434   650106412   
8606  Preferences     preferences      S29420   650106413   
8607  Preferences     preferences      S29422   650106415   
8609  Preferences     preferences      S29423   650106422   
8610  Preferences     preferences      S29430   650106425   
8611  Preferences     preferences      S29421   650106426   
8612  Preferences     preferences      S10173   650106431   
8613  Preferences     preferences      S29470   650106433   
8614  Preferences     preferences      S29469   650106434   
8616  Preferences     preferences       S8299   650106451   
8617  Preferences     preferences       S8300   650106452   
8618  Preferences     preferences       S8301   650106453   
8621  Preferences     preferences       S8310   650106456   
8623  Preferences     preferences       S8324   650106460   
8629  Preferences     preferences       S8349   650106482   
8633  Preferences     preferences       S8364   650106488   
8635  Preferences     preferences       S8365   650106490   
8636  Preferences     preferences       S8367   650106491   
8639  Preferences     preferences       S8338   650106496   
8641  Preferences     preferences       S8340   650106498   
8650  Preferences     preferences       S8410   650106514   
8656  Preferences     preferences      S19144   650106539   
8672  Preferences     preferences      S31544   650106613   
8676  Preferences     preferences       S8501   650106623   
8679  Preferences     preferences       S8504   650106626   
8682  Preferences     preferences       S8508   650106629   
8684  Preferences     preferences       S8513   650106632   
8689  Preferences     preferences       S8524   650106638   
8691  Preferences     preferences       S8526   650106640   
8693  Preferences     preferences      S31830   650106642   
8694  Preferences     preferences      S31826   650106643   
8696  Preferences     preferences      S33904   650106648   
8698  Preferences     preferences      S33905   650106650   
8700  Preferences     preferences      S31796   650106653   
8702  Preferences     preferences      S31818   650106656   
8705  Preferences     preferences      S31794   650106660   
8706  Preferences     preferences      S31803   650106661   
8707  Preferences     preferences      S31813   650106662   
8708  Preferences     preferences      S31811   650106665   
8716  Preferences     preferences      S31799   650106675   
8717  Preferences     preferences      S31802   650106676   
8719  Preferences     preferences      S31836   650106678   
8721  Preferences     preferences      S31840   650106680   
8723  Preferences     preferences      S31832   650106683   
8725  Preferences     preferences      S31835   650106686   
8727  Preferences     preferences      S31844   650106689   
8728  Preferences     preferences      S31841   650106690   
8730  Preferences     preferences      S31831   650106692   
8731  Preferences     preferences      S31

In [30]:
# soy proteins, Keratin Complex Vanilla, Botanical extracts, Hand crafted, Hyaluronic acid, waterproof liner, cocoa and shea butters, Hydrating Lipstick, hydrating, hydrating , Free of sodium chloride, free of sulfates, prevents dandruff,  pain-free,  Silicone-alternative,  breaking up plaque,  preventing tartar buildup,  lifting stains,hyaluronic acid                                                                                        

In [27]:
regex_pattern = r"(?i)(.{50}prevent.{50})|()"
y['matches'] = y['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
z=y[y['matches'].astype(str)!='[]']
print(len(z))
z

209


attributes snake_case_name external_id  product_id  \
636    Preferences     preferences      S36693  1110091582   
646    Preferences     preferences      S36782  1123993544   
647    Preferences     preferences      S36781  1123993545   
961    Preferences     preferences      S36175   888510447   
1144   Preferences     preferences      S35929   829464385   
1220   Preferences     preferences      S35994   814408406   
2194   Preferences     preferences      S21249   650095573   
2455   Preferences     preferences      S22443   650096006   
2467   Preferences     preferences       S1067   650096032   
2470   Preferences     preferences       S1075   650096039   
2483   Preferences     preferences      S18630   650096054   
2544   Preferences     preferences      S19651   650096120   
2547   Preferences     preferences      S22444   650096123   
2554   Preferences     preferences      S19169   650096130   
2569   Preferences     preferences       S1122   650096145   
2572   Preferences     preferences       S1125   650096148   
2987   Preferences     preferences      S18070   650096871   
3019   Preferences     preferences      S19134   650096934   
3037   Preferences     preferences      S19133   650097004   
3143   Preferences     preferences      S19141   650097190   
3291   Preferences     preferences      S18963   650097444   
3307   Preferences     preferences       S1902   650097460   
3323   Preferences     preferences      S30432   650097493   
3326   Preferences     preferences      S25396   650097519   
3805   Preferences     preferences      S24319   650098311   
3831   Preferences     preferences      S20186   650098355   
3923   Preferences     preferences      S23295   650098504   
3928   Preferences     preferences      S23322   650098511   
3930   Preferences     preferences      S23316   650098514   
3932   Preferences     preferences      S23321   650098516   
4041   Preferences     preferences       S3042   650098699   
4054   Preferences     preferences       S3049   650098732   
4838   Preferences     preferences      S23110   650100135   
5572   Preferences     preferences      S18069   650101113   
5573   Preferences     preferences      S20780   650101114   
5574   Preferences     preferences      S18068   650101115   
6118   Preferences     preferences      S28042   650101865   
6708   Preferences     preferences       S6026   650102863   
7035   Preferences     preferences      S33520   650103390   
7199   Preferences     preferences       S6440   650103730   
7478   Preferences     preferences      S19020   650104326   
7745   Preferences     preferences      S30956   650104984   
7790   Preferences     preferences      S18267   650105086   
7844   Preferences     preferences       S7502   650105258   
8321   Preferences     preferences      S29137   650105951   
8358   Preferences     preferences      S24367   650105994   
9752   Preferences     preferences      S10296   650108348   
9765   Preferences     preferences      S10300   650108361   
9767   Preferences     preferences      S10302   650108363   
9799   Preferences     preferences      S19697   650108395   
9800   Preferences     preferences      S19696   650108396   
9987   Preferences     preferences      S32456   650108786   
10332  Preferences     preferences      S18013   650109399   
10399  Preferences     preferences      S17714   650109499   
10411  Preferences     preferences      S17724   650109511   
10456  Preferences     preferences      S11034   650109568   
10698  Preferences     preferences      S18163   650110032   
10714  Preferences     preferences      S29197   650110101   
10770  Preferences     preferences      S11555   650110311   
10778  Preferences     preferences      S11571   650110335   
10804  Preferences     preferences      S11604   650110398   
11020  Preferences     preferences      S11866   650110874   
11286  Preferences     preferences      S27813   650111289   
11287  Preferences     preferences      

In [24]:
regex_pattern = r"(?i)(.{50}free.{50})|()"
y['matches'] = y['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
z=y[y['matches'].astype(str)!='[]']
print(len(z))
print(z['matches'][200:300])

220
18601    [ant superstar that works to protect cells against free-radical damage   Hydrate and protect lips with Ja]
18602    [ant superstar that works to protect cells against free-radical damage   Hydrate and protect lips with Ja]
18758    [arates each lash for perfect definition and clump free wear. Hi-Tech polymers boost strength and volume ]
19454    [asy hydration on the go. Travel-friendly and mess-free, this nourishing liquid lip balm applies like a g]
22029    [ant superstar that works to protect cells against free-radical damage   Hydrate and protect lips with Ja]
22030    [ant superstar that works to protect cells against free-radical damage   Hydrate and protect lips with Ja]
22060    [shape, shine and hydration for a resilient, frizz-free look.  Boosts bounce and definition Adds shine an]
22913    [es Flexible and gentle hybrid brush head for pain-free brushing  Smile brighter with the FOREO ISSA 2 Or]
22914    [es Flexible and gentle hybrid brush head for pain-free bru

In [25]:
regex_pattern = r"(?i)(.{50}help.{50})|()"
y['matches'] = y['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
z=y[y['matches'].astype(str)!='[]']
print(len(z))
print(z['matches'][200:300])

594
7892                                                                                                                                                                                                                         [ayer of dry skin, Oriental Plants Cleansing Cream helps increase product penetration while improving ski]
7917                                                                                                                                                                                                                         [This Volumizing Mascara has a unique formula that helps create the look of gorgeous, full, luxurious las]
7918                                                                                                                                                                                                                         [This Volumizing Mascara has a unique formula that helps create the look of gorgeous, full, luxurious las]
7985        

In [7]:
# Concern_na=df[df['value'].astype(str)=='n/a']
# Concern_na.head(1)

attributes snake_case_name external_id  product_id  \
0    Concern         concern      S37535  1254166673   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
0  This velvety self-tanning mousse raises the bar for self-tanners, as it not only provides a natural-looking bronzed look but does so without the unpleasant odor that's made similar products infamous. Featuring a raspberry almond fragrance and Norvell's Natural Color Complex, this sunless tanner improve sskin&rsquo;s texture &amp; hydration and provides results that rival a spray tan from the comfort of your home. Key Features:  Formulated with breakthrough odor control technology, Ordenone&reg;, designed to virtually eliminate the odor caused by sunless tanning products Addresses signs of aging with tomato seed extract Protects against free radicals with vitamins C and E Soothes and moisturizes with aloe leaf Designed with warm brown with chestnut undertones for a classic, bronze tan  For an instant bronzed look, use Norvell Essentials Self Tanning Mousse.   

  curated_date  customers_id             buckets value resolution  
0   2021-06-08           115  Sun Care & Tanning   n/a   standard

In [29]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

KeyError: 'buckets'

In [13]:
# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# Concern_na['matches']=Concern_na['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #
# Concern_na.head(1)

In [10]:
new_list=['moisture balance',
'dehydration',           
'bleed',
'plaque',
'bacteria',
'gut health',
'touch.?up',
'clumps',
'redness',          
'discoloration',
'under.?eye circles', 
'flyaways',
'skin from damage',
'skin tissue density',
'pores',
'environmental stressors',
'conceal undertones',
'thinning ends',
'excess oil',
'moisture loss',
'complexion',
'complexions',
'remove makeup',
'flake',

          
          
          
'brow shaping',  
'even sunless tan', 
'precision',
'smoothed',
'Dermatologist tested', 
'oil.?free', 

'pores',

'color.?treated hair',

'residue.?free', 
'sulfate.?free',

'golden shimmer',

'bronze',

'smooth application',
'skin from damage',
          'moisture'
         ]

In [11]:
Concern_na.head(1)

attributes snake_case_name external_id  product_id  \
0    Concern         concern      S37535  1254166673   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
0  This velvety self-tanning mousse raises the bar for self-tanners, as it not only provides a natural-looking bronzed look but does so without the unpleasant odor that's made similar products infamous. Featuring a raspberry almond fragrance and Norvell's Natural Color Complex, this sunless tanner improve sskin&rsquo;s texture &amp; hydration and provides results that rival a spray tan from the comfort of your home. Key Features:  Formulated with breakthrough odor control technology, Ordenone&reg;, designed to virtually eliminate the odor caused by sunless tanning products Addresses signs of aging with tomato seed extract Protects against free radicals with vitamins C and E Soothes and moisturizes with aloe leaf Designed with warm brown with chestnut undertones for a classic, bronze tan  For an instant bronzed look, use Norvell Essentials Self Tanning Mousse.   

  curated_date  customers_id             buckets value resolution matches  
0   2021-06-08           115  Sun Care & Tanning   n/a   standard      []

In [14]:
# kwp.add_keywords_from_list(new_list)
# # Extract Key Words
# Concern_na['new_matches']=Concern_na['long_desc'].apply(lambda x: kwp.extract_keywords(x)) 
# new=Concern_na[Concern_na['new_matches'].astype(str)=='[]']

In [15]:
# print(len(new))
# print(new['buckets'].explode().value_counts()[0:10])

In [16]:
# Concern_na[Concern_na['new_matches'].astype(str)!='[]']['new_matches'].explode().value_counts()

In [17]:
# new[2000:2500]

In [ ]:
Concern_na_matches=Concern_na[Concern_na['matches'].astype(str)!='[]']
Concern_na_matches['matches']=Concern_na_matches['matches'].apply(lambda x: sorted(x, key=natural_key))
Concern_na_matches['matches']

In [ ]:
Concern_na_matches['matches']=[list(set(sublist)) for sublist in Concern_na_matches['matches']]
Concern_na_matches['matches']

In [ ]:
Concern_na['matches'].explode().value_counts()

In [ ]:
x

In [ ]:
x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [ ]:
df[df['external_id'].astype(str)=='00739441']

In [ ]:
# x['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [ ]:
split_col(x)

In [ ]:
len(x['matches'].explode().value_counts())

In [ ]:
# error

In [ ]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

In [ ]:
x['matches']=x['matches'].astype(float)

In [ ]:
x

# matches

In [ ]:
matches=x
matches

In [ ]:
matches['matches']=matches['matches'].astype(float)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [ ]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

In [ ]:
matches['Q:outside_diameter'].explode().value_counts()

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)
    

In [ ]:
looks_good('Motion Industries', attribute, df, wipe)